### TIFF TO FITS

run the below cells before doing anything else

In [ ]:
import rawpy
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from astropy.io import fits
import tifffile

In [ ]:
def open_tiff(directory,file):
    tif = cv2.imread(directory+file,cv2.IMREAD_UNCHANGED)

    return cv2.split(tif)



def open_tiff_header(directory,file,metadata_version):
    if metadata_version == "camera":
        with tifffile.TiffFile(directory+file) as tif:
            for page in tif.pages:
                for tag in page.tags:
                    if tag.name == "ExifTag":
                        exifdict = tag.value
                    if tag.name == "Make":
                        model = tag.value
                    if tag.name == "Model":
                        if model in tag.value:
                            model = tag.value
                        else:
                            model = model+" "+tag.value

        metadata = {
                    "CAMMODEL" : model,
                    "CAMLENS" : exifdict["LensModel"],
                    "CAMDATE" : exifdict["DateTimeOriginal"],
                    "CAMEXP" : exifdict["ExposureTime"][0]/exifdict["ExposureTime"][1],
                    "CAMEXPF" : str(exifdict["ExposureTime"][0])+"/"+str(exifdict["ExposureTime"][1]),
                    "CAMFSTOP" : exifdict["FNumber"][0]/exifdict["FNumber"][1],
                    "CAMISO" : exifdict["ISOSpeedRatings"]
                   }
        comments = ["camera model",
                    "camera lens model",
                    "camera digitization date",
                    "camera exposure time (seconds)",
                    "camera exposure time (fractional)",
                    "camera f-stop or f-number",
                    "camera ISO speed"]
        
    elif metadata_version == "epson":
        with tifffile.TiffFile(directory+file) as tif:
            for page in tif.pages:
                for tag in page.tags:
                    if tag.name == "XResolution":
                        metadata = {"SCANDPI" : tag.value[0]}

        comments = ["scanner dots per inch resolution"]
                        
    return metadata,comments



def run_on_directory_rgb(directory,metadata_version):
    files = os.listdir(directory)

    tifffiles = []
    for j in range(len(files)):
        if (files[j][-4:]==".tif") or (files[j][-4:]==".TIF"):
            tifffiles.append(files[j])
    print(f'total of {len(tifffiles)} tiff files in directory')
    
    for i in range(len(tifffiles)):
        blue,green,red = open_tiff(directory,tifffiles[i])
        blue,green,red = np.flip(blue, axis=0),np.flip(green, axis=0),np.flip(red, axis=0)
        
        fits.writeto(directory+tifffiles[i][:-4]+"_r.fits",red,overwrite=True)
        print(f'\nred {tifffiles[i][:-4]} fits saved')
        fits.writeto(directory+tifffiles[i][:-4]+"_g.fits",green,overwrite=True)
        print(f'green {tifffiles[i][:-4]} fits saved')
        fits.writeto(directory+tifffiles[i][:-4]+"_b.fits",blue,overwrite=True)
        print(f'blue {tifffiles[i][:-4]} fits saved')
        
        if metadata_version == "camera" or metadata_version == "epson":
            with fits.open(directory+tifffiles[i][:-4]+"_r.fits", mode='update') as ffile:
                header = ffile[0].header
                dataframe,comments = open_tiff_header(directory,tifffiles[i],metadata_version)
                keywords = list(dataframe.keys())

                for c,key in enumerate(keywords):
                    header[key] = (dataframe[key],comments[c])
            
            with fits.open(directory+tifffiles[i][:-4]+"_g.fits", mode='update') as ffile:
                header = ffile[0].header
                dataframe,comments = open_tiff_header(directory,tifffiles[i],metadata_version)
                keywords = list(dataframe.keys())

                for c,key in enumerate(keywords):
                    header[key] = (dataframe[key],comments[c])
                    
            with fits.open(directory+tifffiles[i][:-4]+"_b.fits", mode='update') as ffile:
                header = ffile[0].header
                dataframe,comments = open_tiff_header(directory,tifffiles[i],metadata_version)
                keywords = list(dataframe.keys())

                for c,key in enumerate(keywords):
                    header[key] = (dataframe[key],comments[c])
    
    print()
    print(f'{len(tifffiles)*3} files saved')

    return

def run_on_directory_g(directory,metadata_version):
    files = os.listdir(directory)

    tifffiles = []
    for j in range(len(files)):
        if (files[j][-4:]==".tif") or (files[j][-4:]==".TIF"):
            tifffiles.append(files[j])
    print(f'total of {len(tifffiles)} tiff files in directory')
    
    for i in range(len(tifffiles)):
        blue,green,red = open_tiff(directory,tifffiles[i])
        blue,green,red = np.flip(blue, axis=0),np.flip(green, axis=0),np.flip(red, axis=0)
        
        fits.writeto(directory+tifffiles[i][:-4]+"_g.fits",green,overwrite=True)
        print(f'\ngreen {tifffiles[i][:-4]} fits saved')
        
        if metadata_version == "camera" or metadata_version == "epson":
            with fits.open(directory+tifffiles[i][:-4]+"_g.fits", mode='update') as ffile:
                header = ffile[0].header
                dataframe,comments = open_tiff_header(directory,tifffiles[i],metadata_version)
                keywords = list(dataframe.keys())

                for c,key in enumerate(keywords):
                    header[key] = (dataframe[key],comments[c])
    
    print()
    print(f'{len(tifffiles)} files saved')

    return

def run_on_directory_gray(directory,convfactors,dtype,metadata_version):
    files = os.listdir(directory)
    rconv,gconv,bconv = convfactors

    tifffiles = []
    for j in range(len(files)):
        if (files[j][-4:]==".tif") or (files[j][-4:]==".TIF"):
            tifffiles.append(files[j])
    print(f'total of {len(tifffiles)} tiff files in directory')
    
    for i in range(len(tifffiles)):
        blue,green,red = open_tiff(directory,tifffiles[i])
        blue,green,red = np.array(blue,dtype=dtype),np.array(green,dtype=dtype),np.array(red,dtype=dtype)
        blue,green,red = np.flip(blue, axis=0),np.flip(green, axis=0),np.flip(red, axis=0)
        data = red*rconv+green*gconv+blue*bconv
        if dtype=="uint16":
            data = np.array(np.rint(data),dtype=dtype)
        else:
            data = np.array(data,dtype=dtype)
        
        fits.writeto(directory+tifffiles[i][:-4]+".fits",data,overwrite=True)
        print(f'\n{tifffiles[i][:-4]} fits saved')
        
        if metadata_version == "camera" or metadata_version == "epson":
            with fits.open(directory+tifffiles[i][:-4]+".fits", mode='update') as ffile:
                header = ffile[0].header
                dataframe,comments = open_tiff_header(directory,tifffiles[i],metadata_version)
                keywords = list(dataframe.keys())

                for c,key in enumerate(keywords):
                    header[key] = (dataframe[key],comments[c])
    
    print()
    print(f'{len(tifffiles)} files saved')

    return

def run_on_file_rgb(directory,file,metadata_version):
    blue,green,red = open_tiff(directory,file)
    blue,green,red = np.flip(blue, axis=0),np.flip(green, axis=0),np.flip(red, axis=0)
    
    fits.writeto(directory+file[:-4]+"_r.fits",red,overwrite=True)
    print(f'red {file[:-4]} fits saved')
    fits.writeto(directory+file[:-4]+"_g.fits",green,overwrite=True)
    print(f'green {file[:-4]} fits saved')
    fits.writeto(directory+file[:-4]+"_b.fits",blue,overwrite=True)
    print(f'blue {file[:-4]} fits saved')
    
    if metadata_version == "camera" or metadata_version == "epson":
        with fits.open(directory+file[:-4]+"_r.fits", mode='update') as ffile:
            header = ffile[0].header
            dataframe,comments = open_tiff_header(directory,file,metadata_version)
            keywords = list(dataframe.keys())

            for c,key in enumerate(keywords):
                header[key] = (dataframe[key],comments[c])

        with fits.open(directory+file[:-4]+"_g.fits", mode='update') as ffile:
            header = ffile[0].header
            dataframe,comments = open_tiff_header(directory,file,metadata_version)
            keywords = list(dataframe.keys())

            for c,key in enumerate(keywords):
                header[key] = (dataframe[key],comments[c])

        with fits.open(directory+file[:-4]+"_b.fits", mode='update') as ffile:
            header = ffile[0].header
            dataframe,comments = open_tiff_header(directory,file,metadata_version)
            keywords = list(dataframe.keys())

            for c,key in enumerate(keywords):
                header[key] = (dataframe[key],comments[c])
    
    return

def run_on_file_g(directory,file,metadata_version):
    blue,green,red = open_tiff(directory,file)
    blue,green,red = np.flip(blue, axis=0),np.flip(green, axis=0),np.flip(red, axis=0)
    
    fits.writeto(directory+file[:-4]+"_g.fits",green,overwrite=True)
    print(f'green {file[:-4]} fits saved')
    
    if metadata_version == "camera" or metadata_version == "epson":
        with fits.open(directory+file[:-4]+"_g.fits", mode='update') as ffile:
            header = ffile[0].header
            dataframe,comments = open_tiff_header(directory,file,metadata_version)
            keywords = list(dataframe.keys())

            for c,key in enumerate(keywords):
                header[key] = (dataframe[key],comments[c])
    
    return

def run_on_file_gray(directory,file,convfactors,dtype,metadata_version):
    blue,green,red = open_tiff(directory,file)
    blue,green,red = np.array(blue,dtype=dtype),np.array(green,dtype=dtype),np.array(red,dtype=dtype)
    blue,green,red = np.flip(blue, axis=0),np.flip(green, axis=0),np.flip(red, axis=0)
    rconv,gconv,bconv = convfactors
    data = red*rconv+green*gconv+blue*bconv
    if dtype=="uint16":
            data = np.array(np.rint(data),dtype=dtype)
    else:
            data = np.array(data,dtype=dtype)
        
    fits.writeto(directory+file[:-4]+".fits",data,overwrite=True)
    print(f'{file[:-4]} fits saved')
    
    if metadata_version == "camera" or metadata_version == "epson":
        with fits.open(directory+file[:-4]+".fits", mode='update') as ffile:
            header = ffile[0].header
            dataframe,comments = open_tiff_header(directory,file,metadata_version)
            keywords = list(dataframe.keys())

            for c,key in enumerate(keywords):
                header[key] = (dataframe[key],comments[c])
    
    return

---

change the below cell to match your preferences, then run the cell

In [ ]:
# change directory to where the file is located, be sure to include the extra slash at the end in the string
# apple users will use / and windows users will use \\
directory = "directory/to/file/"

# file name, used if doing function on only one file
# be sure to include the ending .tif or .TIF (case sensitive!)
file = "file.tif"

# metadata version
# select "camera", "epson", or "none" depending on digitizer ("camera" will work for both Canon and PhaseOne cameras!)
# "none" will append no additional information to the FITS header
metadata_version = "?"



# if converting to grayscale to get one singular FITS file, the following factors are used
# [r,g,b] factors add up to 1.0
convfactors = [1/3,1/3,1/3]

# "uint16", np.float32, np.float64
# np.float32 and np.float64 will return decimals, and "uint16" will return unsigned integers
# ONLY select "uint16" if the rgb convfactors add up to <=1
# np.float64 is higher precision, larger file size than np.float32
# "uint16" will result in roundoff errors, expect values to be up to +/-1 off
dtype = "uint16"

---

run either one or all the cells to save the file(s) to your preference, see hashed descriptions in each cell

In [ ]:
# saves an r,g,b file for all tiff files in the provided directory

run_on_directory_rgb(directory,metadata_version)

In [ ]:
# saves a g file for all tiff files in the provided directory

run_on_directory_g(directory,metadata_version)

In [ ]:
# saves a grayscale converted file for all tiff files in the provided directory

run_on_directory_gray(directory,convfactors,dtype,metadata_version)

In [ ]:
# saves an r,g,b file for the given file and directory

run_on_file_rgb(directory,file,metadata_version)

In [ ]:
# saves a g file for the given file and directory

run_on_file_g(directory,file,metadata_version)

In [ ]:
# saves a grayscale converted file for the given file and directory

run_on_file_gray(directory,file,convfactors,dtype,metadata_version)

---